
---

## 📊 KPIs calculados

| KPI | Descripción |
|-----|-------------|
| `total_transactions` | Total de transacciones en la combinación país + canal |
| `total_amount` | Suma total del dinero transaccionado |
| `avg_amount` | Monto promedio por transacción |
| `fraud_rate_percent` | Porcentaje de transacciones sospechosas (flag) |
| `high_value_ratio_percent` | Porcentaje de transacciones mayores a $10,000 (nivel `high`) |

Estas métricas permiten monitorear:
- Comportamiento por país y canal
- Anomalías por fraude
- Proporción de operaciones de alto valor

---

## 🧊 Destino final

Los datos se almacenan como tabla Delta gestionada



In [0]:
# DBTITLE 1: Imports y configuración
from pyspark.sql.functions import col, count, avg, sum, when, lit, round


# Activar catálogo
try:
    spark.sql(f"USE CATALOG gold_transactions")
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS gold")
    spark.sql(f"USE silver_transactions.silver")

except Exception as e:
    dbutils.notebook.exit(f"[ERROR - GOLD INIT CATALOG AND SCHEMA] {str(e)}")


In [0]:
try:
    storage_account_name = "mistorageprincipal"
    storage_account_key = dbutils.secrets.get("scopeTransaction", "keyTransaction")

    spark.conf.set(
        f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
        storage_account_key
    )
except Exception as e:
    dbutils.notebook.exit(f"[ERROR - GOLD INIT CONFIG] {str(e)}")


In [0]:
try:
    spark.sql("""
        CREATE OR REPLACE TABLE gold_transactions.gold.transactions_general
        USING DELTA
        AS SELECT * FROM silver_transactions.silver.silver_transactions_table
    """)
except Exception as e:
    dbutils.notebook.exit(f"[ERROR - GOLD TRANSACTIONS GENERAL CREATE TABLE] {str(e)}")


In [0]:
spark.sql(
    """
    SELECT * FROM gold_transactions.gold.transactions_general
    """
).display()

In [0]:
try: 
    gold_total_transactions = spark.sql("""
        CREATE OR REPLACE TEMP VIEW gold_total_transactions AS
        SELECT 
            country,
            channel,
            COUNT(*) AS total_transactions
        FROM gold_transactions.gold.transactions_general
        GROUP BY country, channel
    """)
except Exception as e:
    dbutils.notebook.exit(f"[ERROR - GOLD TOTAL TRANSACTIONS TEMP VIEW] {str(e)}")



In [0]:
spark.sql(
    """
        select * from gold_total_transactions
    """
).display()

In [0]:
try:
    spark.sql(
    """
        CREATE OR REPLACE TABLE gold_transactions.gold.total_transactions
        USING DELTA
        AS SELECT * FROM gold_total_transactions
    """
    )
except Exception as e:
    dbutils.notebook.exit(f"[ERROR - GOLD TOTAL TRANSACTIONS CREATE TABLE] {str(e)}")

In [0]:
spark.sql(
    """
        select * from gold_transactions.gold.total_transactions
    """
).display()

In [0]:
try:
    spark.sql("""
        CREATE OR REPLACE TEMP VIEW gold_amounts AS
        SELECT 
            country,
                channel,
            SUM(amount) AS total_amount,
            AVG(amount) AS avg_amount
        FROM gold_transactions.gold.transactions_general
        GROUP BY country, channel
    """)
except Exception as e:
    dbutils.notebook.exit(f"[ERROR - GOLD AMOUNTS TEMP VIEW] {str(e)}")


In [0]:
spark.sql(
    """
    select * from gold_amounts
    """
).display()

In [0]:
try:
    spark.sql(
        """
        CREATE OR REPLACE TABLE gold_transactions.gold.amounts 
        USING DELTA
        SELECT * FROM gold_amounts
        """
    )
except Exception as e:
    dbutils.notebook.exit(f"[ERROR - GOLD AMOUNTS CREATE TABLE] {str(e)}")


In [0]:
try:
    spark.sql("""
        CREATE OR REPLACE TEMP VIEW gold_fraud_rate AS
        SELECT 
            country,
            channel,
            ROUND(
                (SUM(CASE WHEN is_fraud_suspected = true THEN 1 ELSE 0 END) * 100.0) / COUNT(*), 2
            ) AS fraud_rate_percent
        FROM gold_transactions.gold.transactions_general
        GROUP BY country, channel
    """)
except Exception as e:
    dbutils.notebook.exit(f"[ERROR - GOLD FRAUD RATE TEMP VIEW] {str(e)}")


In [0]:
spark.sql(
    """
    select * from gold_fraud_rate
    """
).display()

In [0]:
try:
    spark.sql(
    """
    CREATE OR REPLACE TABLE gold_transactions.gold.fraud_rate 
    USING DELTA
    SELECT * FROM gold_fraud_rate
    """
)
except Exception as e:
    dbutils.notebook.exit(f"[ERROR - GOLD FRAUD RATE CREATE TABLE] {str(e)}")

In [0]:
try:
    spark.sql("""
        CREATE OR REPLACE TEMP VIEW gold_high_value_ratio AS
        SELECT 
            country,
            channel,
            ROUND(
                (SUM(CASE WHEN amount_level = 'high' THEN 1 ELSE 0 END) * 100.0) / COUNT(*), 2
            ) AS high_value_ratio_percent
        FROM gold_transactions.gold.transactions_general
        GROUP BY country, channel
    """)
except Exception as e:
    dbutils.notebook.exit(f"[ERROR - GOLD HIGH VALUE RATIO TEMP VIEW] {str(e)}")

In [0]:
spark.sql(
    """
    select * from gold_high_value_ratio
    """
).display()

In [0]:
try:
    spark.sql(
        """
            CREATE OR REPLACE TABLE gold_transactions.gold.high_value_ratio
            USING DELTA 
            SELECT * FROM gold_high_value_ratio
        """
    )
except Exception as e:
    dbutils.notebook.exit(f"[ERROR - GOLD HIGH VALUE RATIO CREATE TABLE] {str(e)}")

### Unión final de todas las _métricas_

In [0]:
try:
    spark.sql("""
        CREATE OR REPLACE TEMP VIEW global_kpis AS
        SELECT 
            a.country,
            a.channel,
            a.total_transactions,
            b.total_amount,
            b.avg_amount,
            c.fraud_rate_percent,
            d.high_value_ratio_percent
        FROM gold_total_transactions a
        JOIN gold_amounts b ON a.country = b.country AND a.channel = b.channel
        JOIN gold_fraud_rate c ON a.country = c.country AND a.channel = c.channel
        JOIN gold_high_value_ratio d ON a.country = d.country AND a.channel = d.channel
    """)
except Exception as e:
    dbutils.notebook.exit(f"[ERROR - GOLD GLOBAL KPIS TEMP VIEW] {str(e)}")

In [0]:
spark.sql(
    """
    select * from global_kpis
    """
).display()

### Escribir como tabla Delta en Gold

In [0]:
try:
    spark.conf.set(
        f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
        storage_account_key
    )

    storage_account_key = dbutils.secrets.get("scopeTransaction", "keyTransaction")

    spark.conf.set(
        f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
        storage_account_key
    )
except Exception as e:
    dbutils.notebook.exit(f"[ERROR - LAST SETS FS] {str(e)}")



In [0]:
try:
    spark.sql("USE CATALOG gold_transactions")
    spark.sql("CREATE SCHEMA IF NOT EXISTS gold")

    spark.sql("""
        CREATE OR REPLACE TABLE gold_transactions.gold.global_kpis
        USING DELTA
        AS SELECT * FROM global_kpis
    """)
except Exception as e:
    dbutils.notebook.exit(f"[ERROR - GOLD KPIS CREATE TABLE] {str(e)}")

try:
# Guardo global_kpis en una variable df
    global_kpis_df = spark.table("global_kpis")
    global_general_df = spark.table("gold_transactions.gold.transactions_general")

# Escribir en CSV
    global_kpis_df.write.mode("overwrite").csv("abfss://csvfinalsmartbank@mistorageprincipal.dfs.core.windows.net/final_csv_enriched_transactions_kpis/", header=True)
    global_general_df.write.mode("overwrite").csv("abfss://csvfinalsmartbank@mistorageprincipal.dfs.core.windows.net/final_csv_general_transactions/", header=True)

except Exception as e:
    dbutils.notebook.exit(f"[ERROR - WRITE CSV] {str(e)}")
    
dbutils.notebook.exit("OK")
